#### Case 2: Liquid hydrogen ($LH_2$) in a 2033 $\text{m}^3$ tank

This application corresponds to liquid hydrogen storage in the NASA's Space Launch System. It consists of a 2033 $m^3$ storage tank [NASA](https://www.energy.gov/sites/default/files/2021-10/new-lh2-sphere.pdf) in a tank with 8.4 m diameter and 40 m height. It will be assumed the following operation scenarios:

* Daily boil-off rate of 0.1%  
* Storage at atmospheric pressure with continuous removal of boil-off gas

For purposes of the example, it is assumed that the storage tank is passively insulated with perlite.

03/09/2024: Illustration of how to use SciPy to optimise

In [ ]:
# Scientific computing
import numpy as np

# Visualisation
import matplotlib.pyplot as plt

# Import the storage tank Class
from cryoevap.storage_tanks import Tank

# Import Cryogen class
from cryoevap.cryogens import Cryogen

#### Setup tank and cryogen properties


In [ ]:
# Cylindrical storage tank properties
Q_roof = 0 # Roof heat ingress / W

# Vertically orientated cylindrical tank volume
V_tank = 2033/100 #m^3

# aspect ratio
a = 0.5

d_i = (4*V_tank/(a*np.pi))**(1/3) # Internal diameter / m
d_o = d_i + 0.04   # External diameter / m
T_air = 293.15 # Temperature of the environment K

# Set overall heat transfer coefficient through the walls for liquid and vapour
U_L = 3.73e-3 # W/m2/K
U_V = 3.73e-3 # W/m2/K

# Specify heat transfer rate at the bottom
# This will represent the heat conduction from the piping system
Q_b = 10 # 100 # W, 


#V_tank = 500 #m^3

# Initial liquid filling / Dimensionless
LF = 0.95 

# Specify tank operating pressure
P = 101325 # Pa

eta_w = 0.9 # Evaporative fraction

# Initialize mid-scale tank
mid_tank = Tank(d_i, d_o, V_tank, LF)
mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_b, Q_roof, eta_w= eta_w)

# Keep the tank roof insulated
mid_tank.U_roof = 0 #U_V * (1-eta_w)

# Initialise cryogen
hydrogen = Cryogen(name = "hydrogen")
hydrogen.set_coolprops(P)

# Set cryogen
mid_tank.cryogen = hydrogen

Calculate initial evaporation rate and transient period

In [ ]:
# Calculate initial evaporation rate
print("The initial evaporation rate of " + hydrogen.name + " is %.1f kg/h" % (mid_tank.b_l_dot * 3600))

# Estimate transient period duration
print("Transient period = %.3f s " % mid_tank.tau)

# Minimum number of hours to achieve steady state 
tau_h = (np.floor(mid_tank.tau / 3600) + 1)

# Print simulation time of the transient period for short-term storage
print("Simulation time: %.0i h" % tau_h )

# Calculate boil-off rate
BOR = (mid_tank.b_l_dot * 24 * 3600) / (mid_tank.V * mid_tank.LF * mid_tank.cryogen.rho_L)
print("BOR = %.3f %%" % (BOR * 100))

### Simulation setup and execution

### Define optimisation parameters

In [ ]:
# Define vertical spacing
dz = 0.04

# Calculate number of nodes
n_z = 1 + int(np.round(mid_tank.l_V/dz, 0))

# Define dimensionless computational grid
mid_tank.z_grid = np.linspace(0, 1, n_z)

# Define evaporation time as the transient period
evap_time = 3600 * 48

# evap_time = 3600
# Time step to plot each vapour temperature profile
mid_tank.plot_interval = evap_time/6

# Time step to record data, relevant for plotting integrated quantities such as
# the vapour to liquid heat transfer rate, Q_VL
mid_tank.time_interval = 60

mid_tank.evaporate(evap_time)

### Plot response surface

In [ ]:
# Liquid filling
LF_vec = np.linspace(0.05, 0.95, 9)

# BOR objective function
f_list = []
# Total BOG objective function
f2_list = []

for LFi in LF_vec:
    # Update liquid filling
    mid_tank.LF = LFi

    # Execute simulation
    mid_tank.evaporate(evap_time)

    # Calculate objective function
    f = 1 - mid_tank.data['V_L'][-1] / mid_tank.data['V_L'][0]

    f2 = mid_tank.data['V_L'][0] - mid_tank.data['V_L'][1]

    # Append to list
    f_list.append(f)
    f2_list.append(f2)

f_list = np.array(f_list)
f2_list = np.array(f2_list)

fig, ax = plt.subplots(1,2)
# Plot
ax[0].plot(LF_vec, (100*f_list), 'ro')
ax[0].set_xlabel('LF')
ax[0].set_ylabel('BOR')
    
ax[1].plot(LF_vec, f2_list, 'ro')
ax[1].set_xlabel('LF')
ax[1].set_ylabel('BOG')

### Aspect ratio

In [ ]:
# Aspect ratio
a_vec = np.linspace(0.1, 2, 7)

# BOR objective function
f_list = []
# Total BOG objective function
f2_list = []

for a in a_vec:
    # Update aspect ratio
    # a = 0.5
    d_i = (4*V_tank/(a*np.pi))**(1/3) # Internal diameter / m
    mid_tank.d_i = d_i
    
    mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_b, Q_roof, eta_w= eta_w)

    # Execute simulation
    mid_tank.evaporate(evap_time)

    # Calculate objective function
    f = 1 - mid_tank.data['V_L'][-1] / mid_tank.data['V_L'][0]

    f2 = mid_tank.data['V_L'][0] - mid_tank.data['V_L'][-1]

    # Append to list
    f_list.append(f)
    f2_list.append(f2)

f_list = np.array(f_list)
f2_list = np.array(f2_list)

fig, ax = plt.subplots(1,2)
# Plot
ax[0].plot(a_vec, (100*f_list), 'ro')
ax[0].set_xlabel('a=L/D')
ax[0].set_ylabel('BOR')
    
ax[1].plot(a_vec, f2_list, 'ro')
ax[1].set_xlabel('a=L/D')
ax[1].set_ylabel('BOG')

In [ ]:
from scipy.optimize import Bounds, minimize

# Minimum and maximum practical ranges
# of initial liquid filling

# 5% represent ballast voyage
# 95% is the safety limit for potential liquid thermal expansion
bounds = Bounds([0.05], [0.95])

# Define objective function
def BOR_function(LF):

    # Update liquid filling
    mid_tank.LF = LF

    # Execute simulation
    mid_tank.evaporate(evap_time)

    # Calculate objective function
    f = 1 - mid_tank.data['V_L'][-1] / mid_tank.data['V_L'][0]
    
    print("LF = %.3f, f=%.3e" % (LF, f))
    return f    

# Initial liquid filling to optimise

x0 = 0.9 
res = minimize(BOR_function, x0, method='trust-constr', options={'verbose': 1}, bounds=bounds)

Global optimisation

In [ ]:
from scipy.optimize import Bounds, differential_evolution

# Minimum and maximum practical ranges
# of initial liquid filling
# 5% represent ballast voyage
# 95% is the safety limit for potential liquid thermal expansion
bounds = [(0.05, 0.95)]  # Bounds in differential_evolution are specified as a list of tuples

# Define objective function
def BOR_function(LF):

    # Update liquid filling
    mid_tank.LF = LF[0]  # LF is now an array, take the first element

    # Execute simulation
    mid_tank.evaporate(evap_time)

    # Calculate objective function
    
    # Liquid volume aet the end of the evaporation
    V_LF = mid_tank.data['V_L'][-1]

    # Liquid volume at the beginning
    V_L0 = mid_tank.data['V_L'][0]

    # Objective function
    f = 1 - V_LF/ V_L0
    
    print("LF = %.3f, V_L0=%.3f, V_LF = %.3f,, f=%.3e" % (LF[0], V_L0, V_LF, f))
    return f    

# Perform global optimization using differential evolution
result = differential_evolution(BOR_function, bounds, strategy='best1bin', maxiter=1000, popsize=15, tol=1e-6, mutation=(0.5, 1), recombination=0.7, disp=True)

# Results
print("Best solution found: LF =", result.x[0])
print("Function value:", result.fun)


### Visualisation of results

#### Vapour temperature

In [ ]:
mid_tank.plot_tv(t_unit='h')

Visualise liquid and vapour heat ingresses, $\dot{Q}_{\text{L}}$ and  $\dot{Q}_{\text{V}}$.

The plot also shows the vapour to liquid heat ingress, $\dot{Q}_{VL}$, and  the partition of the vapour heat ingress that is transferred to the interface by the wall directly, $\dot{Q}_{\text{V,w}}$

In [ ]:
# Specify y-axis units as W, and time units to hours
mid_tank.plot_Q(unit = 'W', t_unit = 'h')

#### Plot liquid volume

In [ ]:
mid_tank.plot_V_L(t_unit='h')

#### Plot evaporation rate, $\dot{B}_{\text{L}}$, and boil-off gas rate, $\dot{B}_{}$

In [ ]:
mid_tank.plot_BOG(unit='kg/h', t_unit='h')

Optional: CSV data export

If evaporation data is intended to be post-processed in another software, it can be exported readily with the help of the Pandas package.

In [ ]:
# Import pandas 
import pandas as pd

Plot average vapour and boil-off gas temperature

In [ ]:
mid_tank.plot_tv_BOG(t_unit='h')

#### References

U.S. Department of Energy. (2021, October). DOE/NASA Advances in Liquid Hydrogen Storage Workshop. Retrieved from [https://www.energy.gov/sites/default/files/2021-10/new-lh2-sphere.pdf]